<a href="https://colab.research.google.com/github/Hoiy/kaggle-jigsaw-unintended-bias-in-toxicity-classification/blob/master/kaggle_jigsaw_unintended_bias_in_toxicity_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.
  

import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import tokenization
from run_classifier import *
import pandas as pd
from tqdm import tqdm

In [0]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'kaggle-195702'
!gcloud config set project {project_id}
!mkdir -p assets
!gsutil -m rsync -rd gs://kaggle-195702-jigsaw-unintended-bias-in-toxicity-classification/assets ./assets

In [0]:
class JigsawUBITCProcessor(DataProcessor):
    def get_train_examples(self, data_dir):
        df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
        train = df.sample(frac=0.8, random_state=42)
        df = train
        pos = df[df.target >= 0.5]
        neg = df[df.target < 0.5].sample(n=len(pos))
        df = pd.concat([pos, neg]).sample(frac=1)
        return self._create_examples(df, "train")

    def get_dev_examples(self, data_dir):
        df = pd.read_csv(os.path.join(data_dir, 'train.csv'))
        train = df.sample(frac=0.8, random_state=42)
        df = df[~df.index.isin(train.index)]
        pos = df[df.target >= 0.5]
        neg = df[df.target < 0.5].sample(n=len(pos))
        df = pd.concat([pos, neg]).sample(frac=1)
        return self._create_examples(df, "train")

    def get_test_examples(self, data_dir):
        return self._create_examples(
            pd.read_csv(os.path.join(data_dir, 'test.csv')), "test")

    def get_labels(self):
        return ["0", "1"]

    def _create_examples(self, df, set_type):
        examples = []
        if set_type == 'train':
            df = df.sample(frac=1).reset_index(drop=True)

        for i, row in tqdm(df.iterrows(), total=len(df)):
            guid = "%s-%s" % (set_type, i)
            if set_type == "test":
                text_a = tokenization.convert_to_unicode(row['comment_text'])
                label = "0"
            else:
                text_a = tokenization.convert_to_unicode(row['comment_text'])
                label = "1" if row['target'] >= 0.5 else "0"
            examples.append(
              InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

In [0]:
class flags(object):
    do_lower_case = True
    max_seq_length = 256
    output_dir = 'gs://kaggle-195702-jigsaw-unintended-bias-in-toxicity-classification/output'
    dataset_dir = 'gs://kaggle-195702-jigsaw-unintended-bias-in-toxicity-classification/dataset'
    use_tpu = True
    tpu_name = TPU_ADDRESS
    tpu_zone = None
    gcp_project = None
    init_checkpoint = 'gs://kaggle-195702-jigsaw-unintended-bias-in-toxicity-classification/assets/wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt'
    bert_config_file = 'gs://kaggle-195702-jigsaw-unintended-bias-in-toxicity-classification/assets/wwm_uncased_L-24_H-1024_A-16/bert_config.json'
    vocab_file = 'gs://kaggle-195702-jigsaw-unintended-bias-in-toxicity-classification/assets/wwm_uncased_L-24_H-1024_A-16/vocab.txt'
    data_dir = './assets'
    master = None
    save_checkpoints_steps = 3000
    iterations_per_loop = 1000
    num_tpu_cores = None
    do_train = True
    do_eval = True
    do_pred = False
    learning_rate = 1e-7
    train_batch_size = 80
    eval_batch_size = 80
    predict_batch_size = 8
    warmup_proportion = 0.0
    num_train_epochs = 1.

    
FLAGS = flags()

tf.logging.set_verbosity(tf.logging.INFO)

tokenization.validate_case_matches_checkpoint(FLAGS.do_lower_case,
                                              FLAGS.init_checkpoint)

if not FLAGS.do_train and not FLAGS.do_eval and not FLAGS.do_predict:
  raise ValueError(
      "At least one of `do_train`, `do_eval` or `do_predict' must be True.")

bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

if FLAGS.max_seq_length > bert_config.max_position_embeddings:
  raise ValueError(
      "Cannot use sequence length %d because the BERT model "
      "was only trained up to sequence length %d" %
      (FLAGS.max_seq_length, bert_config.max_position_embeddings))

tf.gfile.MakeDirs(FLAGS.output_dir)

processor = JigsawUBITCProcessor()

label_list = processor.get_labels()

tokenizer = tokenization.FullTokenizer(
    vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

tpu_cluster_resolver = None
if FLAGS.use_tpu and FLAGS.tpu_name:
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(
      FLAGS.tpu_name, zone=FLAGS.tpu_zone, project=FLAGS.gcp_project)

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    master=FLAGS.master,
    model_dir=FLAGS.output_dir,
    save_summary_steps = 1000,
    keep_checkpoint_max = 99999,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=FLAGS.iterations_per_loop,
        num_shards=FLAGS.num_tpu_cores,
        per_host_input_for_training=is_per_host))

# num_train_examples = 1443889
# TRAIN_FILE_NAME = 'train_80.tf_record'

# num_train_examples = 1804874
# TRAIN_FILE_NAME = 'train.tf_record'

num_train_examples = 3321080
TRAIN_FILE_NAME = 'train_upsample.tf_record'



train_examples = None
num_train_steps = None
num_warmup_steps = None
if FLAGS.do_train:
#   train_examples = processor.get_train_examples(FLAGS.data_dir)
  num_train_steps = int(
      num_train_examples / FLAGS.train_batch_size * FLAGS.num_train_epochs)
  num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)
  
train_examples, num_train_steps, num_warmup_steps

In [0]:
model_fn = model_fn_builder(
    bert_config=bert_config,
    num_labels=len(label_list),
    init_checkpoint=FLAGS.init_checkpoint,
    learning_rate=FLAGS.learning_rate,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=FLAGS.use_tpu,
    use_one_hot_embeddings=FLAGS.use_tpu)

# If TPU is not available, this will fall back to normal Estimator on CPU
# or GPU.
estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=FLAGS.use_tpu,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=FLAGS.train_batch_size,
    eval_batch_size=FLAGS.eval_batch_size,
    predict_batch_size=FLAGS.predict_batch_size)

if FLAGS.do_train:
  train_file = os.path.join(FLAGS.dataset_dir, TRAIN_FILE_NAME)
#   file_based_convert_examples_to_features(
#       train_examples, label_list, FLAGS.max_seq_length, tokenizer, train_file)
  tf.logging.info("***** Running training *****")
  tf.logging.info("  Num examples = %d", num_train_examples)
  tf.logging.info("  Batch size = %d", FLAGS.train_batch_size)
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = file_based_input_fn_builder(
      input_file=train_file,
      seq_length=FLAGS.max_seq_length,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

if FLAGS.do_eval:
  eval_examples = processor.get_dev_examples(FLAGS.data_dir)
  num_actual_eval_examples = len(eval_examples)
  if FLAGS.use_tpu:
    # TPU requires a fixed batch size for all batches, therefore the number
    # of examples must be a multiple of the batch size, or else examples
    # will get dropped. So we pad with fake examples which are ignored
    # later on. These do NOT count towards the metric (all tf.metrics
    # support a per-instance weight, and these get a weight of 0.0).
    while len(eval_examples) % FLAGS.eval_batch_size != 0:
      eval_examples.append(PaddingInputExample())

  eval_file = os.path.join(FLAGS.output_dir, "eval.tf_record")
  file_based_convert_examples_to_features(
      eval_examples, label_list, FLAGS.max_seq_length, tokenizer, eval_file)

  tf.logging.info("***** Running evaluation *****")
  tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                  len(eval_examples), num_actual_eval_examples,
                  len(eval_examples) - num_actual_eval_examples)
  tf.logging.info("  Batch size = %d", FLAGS.eval_batch_size)

  # This tells the estimator to run through the entire set.
  eval_steps = None
  # However, if running eval on the TPU, you will need to specify the
  # number of steps.
  if FLAGS.use_tpu:
    assert len(eval_examples) % FLAGS.eval_batch_size == 0
    eval_steps = int(len(eval_examples) // FLAGS.eval_batch_size)

  eval_drop_remainder = True if FLAGS.use_tpu else False
  eval_input_fn = file_based_input_fn_builder(
      input_file=eval_file,
      seq_length=FLAGS.max_seq_length,
      is_training=False,
      drop_remainder=eval_drop_remainder)

  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)

  output_eval_file = os.path.join(FLAGS.output_dir, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    tf.logging.info("***** Eval results *****")
    for key in sorted(result.keys()):
      tf.logging.info("  %s = %s", key, str(result[key]))
      writer.write("%s = %s\n" % (key, str(result[key])))